In [ ]:
"""
Economist Application Pt. 1 -- Cameron Ferwerda

Your script should output two CSVs, called polls.csv and trends.csv (for the averages). The polls file should have columns for 
date, pollster, n (sample size), and each candidate (by name); the trends file just date and a column for each candidate. Values 
for polls and trends should be a number from 0 to 1. The polls file should have a row for each poll. The trends file should 
have a row for each day, starting with October 11th, 2023.

Ideally it will have tests or some sort of error monitoring, and will detect or alert us in the event of a major error. 
(Writing errors to a log file is a sufficient alert for this assignment.) Your entire program should be executable from 
a single command, but it probably shouldn’t be written in a single monolithic script.
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.patches
import bambi as bmb
import arviz as az
import logging
import datetime as dt
from econ_app_utils import plot_predictions, export_trend

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

sunlight_cat = ["#156B90", "#9a3e25", "#708259","#bd8f22" ,"#842854","#ba5f06","#0f8c79","#bd2d28","#A0B700","#f2da57","#8e6c8a","#7abfcc", "#f3a126"]

sns.color_palette(sunlight_cat)

dict_rc = {"grid.color": "#efecea", 'axes.facecolor':"#efecea", 'figure.facecolor':"#efecea", 'axes.edgecolor': "#635F5D",

               'axes.labelcolor': "#635F5D", 'text.color': '#635F5D','xtick.color': '#635F5D','ytick.color': '#635F5D'}; # "grid.color": "#635F5D",

sns.set_theme(context='notebook', palette = sunlight_cat, rc = dict_rc)

In [ ]:
df_list = pd.read_html('https://cdn-dev.economistdatateam.com/jobs/pds/code-test/index.html')
df = df_list[0]

In [ ]:
cols = df.columns

In [ ]:
df_ind = df.iloc[:,: 3]
df_cand = df.iloc[:,-(len(cols) - len(df_ind.columns)):]

In [ ]:
df_cand = df_cand.replace('%','',regex=True).replace('**',np.nan)

In [ ]:
try:
    df_cand = df_cand.astype(float)
    df_cand = df_cand / 100
except:
    logging.basicConfig(level=logging.INFO,filename='err_log.txt',filemode='a')
    logging.error('Exception: ', exc_info=True)

In [ ]:
df_merge = df_ind.merge(df_cand, right_index=True, left_index=True)

In [ ]:
df_merge.Date = pd.to_datetime(df_merge.Date)

In [ ]:
df_resample = df_merge.set_index('Date').resample('D').mean().reset_index()

In [ ]:
df_resample = df_resample.reset_index().rename(columns={'index':'count_idx'})

In [ ]:
df_resample.head()

In [ ]:
df_melt_rs = pd.melt(df_resample, id_vars=['Date','count_idx'], value_vars=df_cand.columns, var_name='Candidate', value_name='Proportion')

In [ ]:
levels_rs, categories_rs = pd.factorize(df_melt_rs['Candidate'])
colors_rs = [sunlight_cat[i] for i in levels_rs]
handles_rs = [matplotlib.patches.Patch(color=sunlight_cat[i], label=c) for i, c in enumerate(categories_rs)]

In [ ]:
df_melt_rs.plot.scatter(x='Date',y='Proportion',c=colors_rs)
plt.legend(handles=handles_rs,loc='upper right')

In [ ]:
df_list = []

for cand in df_cand.columns:
    df_list.append(df_melt_rs.groupby('Candidate').get_group(cand).dropna())

In [ ]:
num_knots = 11

knot_list = []

for i in range(len(df_list)):
    knot_list.append(np.quantile(df_list[i]['count_idx'], np.linspace(0, 1, num_knots)))

In [ ]:
iknots_list = []

for i in range(len(knot_list)):
    iknots_list.append(knot_list[i][1:-1])

In [ ]:
knot_dict = {}

for i in range(len(knot_list)):
    knot_dict["iknots_{}".format(i)] = knot_list[i][1:-1]

In [ ]:
for i in range(len(knot_list)):
    exec(f'iknots_{i} = knot_list[i][1:-1]')

In [ ]:
"""
y ~ N(mu, sigma)
mu = a + sum(w*B)
"""

priors = {
    "Intercept": bmb.Prior("Normal", mu=0, sigma=1),
    "Weight": bmb.Prior("Normal", mu=0, sigma=1), 
    "Sigma": bmb.Prior("Exponential", lam=1)
}

model_list = []

for i in range(len(df_list)):
    model_list.append(bmb.Model(f'Proportion ~ bs(count_idx, knots=iknots_{i}, intercept=True)', data=df_list[i], priors=priors, dropna=True))

In [ ]:
fit_list = []

for i in range(len(model_list)):
    fit_list.append(model_list[i].fit(inference_method='mcmc',draws=1000,tune=1000,chains=2,idata_kwargs={"log_likelihood": True}))

In [ ]:
def exp_csv(data, idata, model, dates):
    
    # Create a test dataset with observations spanning the whole range of year
    new_data = pd.DataFrame({"count_idx": np.linspace(dates.count_idx.min(), dates.count_idx.max(), num=len(dates))})

    # Predict
    posterior_stacked = []
    y_hat = []
    y_hat_mean = []
    hdi_data = []
    for i in range(len(idata)):
        model[i].predict(idata[i], data=new_data)
        posterior_stacked.append(az.extract_dataset(idata[i]))
        y_hat.append(posterior_stacked[i]["Proportion_mean"])
        y_hat_mean.append(y_hat[i].mean('sample'))
        hdi_data.append(np.quantile(y_hat[i], [0.03, 0.97], axis=1))

    df_trend_list = []

    for i in range(len(idata)):
        df_trend_list.append(pd.DataFrame({'Date':dates['Date'],
                        f'{data[i].Candidate.unique()[0]}_y_hat_mean':y_hat_mean[i],
                        f'{data[i].Candidate.unique()[0]}_lower_bound':hdi_data[i][0],
                        f'{data[i].Candidate.unique()[0]}_upper_bound':hdi_data[i][1]}))

    df_concat = pd.concat(df_trend_list,axis=1)

    df_concat = df_concat.loc[:,~df_concat.columns.duplicated()].copy()

    df_concat.to_csv('trends.csv',index=False)

    return

In [ ]:
exp_csv(df_list,fit_list,model_list,df_resample)

In [ ]:
for i in range(len(fit_list)):
    plot_predictions(df_list[i], fit_list[i], model_list[i], knot_list[i], df_resample);

In [ ]:
export_trend(df_list, fit_list, model_list, df_resample)